# Function Calling

Function calling allows you to define functions (with schemas) that the model can call as part of its response, enabling structured outputs, tool use, and more advanced workflows. 

For us, this seems like a first step towards our goal of enabling more complex interactions with LLMs, where they can not only generate text but also perform actions based on that text.

## Prerequisites

Make sure you have your OpenAI API key set up as described in the previous notebook. We'll use the Python SDK for these examples.

In [8]:
# VS Code's Jupyter extension doesn't support loading .envrc, so if you're using VS Code, we load it here.

from utils import load_envrc

load_envrc()

## Defining a function schema

We need to define a function that the model can use. Here we'll use a silly example function that "cactifies" a name. The schema for the function is defined in [JSON Schema format](https://json-schema.org/docs) and looks like this:

```json
{
  "name": "cactify_name",
  "type": "function",
  "description": "Transforms a name into a fun, cactus-themed version.",
  "parameters": {
    "type": "object",
    "properties": {
      "name": {
        "type": "string",
        "description": "The name to be cactified."
      }
    },
    "required": ["name"]
  }
}
```

Following the [OpenAI function calling docs](https://platform.openai.com/docs/guides/function-calling), we can pass a `tools` parameter to the API endpoint to define functions the model can call. Here we use `curl` to demonstrate the raw API request:

In [9]:
%%bash --out curl_response

curl https://api.openai.com/v1/responses -s \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -d '{
    "model": "gpt-5-nano",
    "input": [
      {"role": "user", "content": "What would my name, Colin, be if it were cactus-ified?"}
    ],
    "tools": [
      {
        "name": "cactify_name",
        "type": "function",
        "description": "Transforms a name into a fun, cactus-themed version.",
        "parameters": {
          "type": "object",
          "properties": {
            "name": {
              "type": "string",
              "description": "The name to be made cactus-like."
            }
          },
          "required": ["name"]
        }
      }
    ]
  }'

In [10]:
import json

from rich import print as rich_print

data = json.loads(curl_response)  # noqa
rich_print(data["output"])

[
    {'id': 'rs_0f6c0809e42d76950068c9d7930b748197a68348c3650098f5', 'type': 'reasoning', 'summary': []},
    {
        'id': 'fc_0f6c0809e42d76950068c9d793d714819793a8a604b41f8a10',
        'type': 'function_call',
        'status': 'completed',
        'arguments': '{"name":"Colin"}',
        'call_id': 'call_eckxIyxX7yoEpszYMLzCblmx',
        'name': 'cactify_name'
    }
]

Reviewing the output, we see that the model decided to call our `cactify_name` function with the argument `"Colin"`. The model itself doesn't actually execute the function. It simply returns the function call in its response. It's up to us to handle the function execution and return the result if needed.

It's easier to see this in action using the Python SDK, which we'll explore next.

## Using the Python SDK

Now let's see how to do the same thing using the Python SDK. We'll define the function schema as a variable:

In [12]:
cactify_name_schema = {
    "name": "cactify_name",
    "type": "function",
    "description": "Transform a name into a fun, cactus-themed version.",
    "parameters": {
        "type": "object",
        "properties": {"name": {"type": "string", "description": "The name to be cactified."}},
        "required": ["name"],
    },
}

## Making a function call request

Now, let's ask the model to cactify a name using the function we defined:

In [13]:
import openai

client = openai.Client()

input_list = [{"role": "user", "content": "What would my name, Colin, be if it were cactus-ified?"}]
tools = [cactify_name_schema]

response = client.responses.create(
    model="gpt-5-nano",
    input=input_list,
    tools=tools,
)
rich_print(response)

Response(
    id='resp_07760e000eb25cdc0068c9d79c004481a3a09c1dfaefe39d87',
    created_at=1758058396.0,
    error=None,
    incomplete_details=None,
    instructions=None,
    metadata={},
    model='gpt-5-nano-2025-08-07',
    object='response',
    output=[
        ResponseReasoningItem(
            id='rs_07760e000eb25cdc0068c9d79d790881a3a2054d2db49f34ae',
            summary=[],
            type='reasoning',
            content=None,
            encrypted_content=None,
            status=None
        ),
        ResponseFunctionToolCall(
            arguments='{"name":"Colin"}',
            call_id='call_y8ouIrJZrcKo0BCIP8UCSuVK',
            name='cactify_name',
            type='function_call',
            id='fc_07760e000eb25cdc0068c9d79e50ac81a3b4273b683a1e7e1d',
            status='completed'
        )
    ],
    parallel_tool_calls=True,
    temperature=1.0,
    tool_choice='auto',
    tools=[
        FunctionTool(
            name='cactify_name',
            parameters={
                'type': 'object',
                'properties': {'name': {'type': 'string', 'description': 'The name to be cactified.'}},
                'required': ['name'],
                'additionalProperties': False
            },
            strict=True,
            type='function',
            description='Transform a name into a fun, cactus-themed version.'
        )
    ],
    top_p=1.0,
    background=False,
    conversation=None,
    max_output_tokens=None,
    max_tool_calls=None,
    previous_response_id=None,
    prompt=None,
    prompt_cache_key=None,
    reasoning=Reasoning(effort='medium', generate_summary=None, summary=None),
    safety_identifier=None,
    service_tier='default',
    status='completed',
    text=ResponseTextConfig(format=ResponseFormatText(type='text'), verbosity='medium'),
    top_logprobs=0,
    truncation='disabled',
    usage=ResponseUsage(
        input_tokens=73,
        input_tokens_details=InputTokensDetails(cached_tokens=0),
        output_tokens=151,
        output_tokens_details=OutputTokensDetails(reasoning_tokens=128),
        total_tokens=224
    ),
    user=None,
    store=True
)

Like the `curl` example, the response includes a function call with arguments. Let's extract and run it.

In [14]:
import json


def cactify_name(name: str) -> str:
    """
    Makes a name more cactus-like by adding or replacing the end with 'ctus'.

    Args:
        name: The name to be cactified.

    Returns:
        The cactified version of the name.
    """

    base_name = name

    # Rule 1: If the name ends in 's' or 'x', remove it.
    # Example: "James" -> "Jame", "Alex" -> "Ale"
    if base_name.lower().endswith(("s", "x")):
        base_name = base_name[:-1]

    # Rule 2: If the name now ends in a vowel, remove it.
    # Example: "Jame" -> "Jam", "Mike" -> "Mik", "Anna" -> "Ann"
    if base_name and base_name.lower()[-1] in "aeiou":
        base_name = base_name[:-1]

    # Add the smoother suffix
    return base_name + "actus"


response_output = response.output
# The model decided to call our function
function_call = response.output[1]
# Load the arguments provided by the model to call the function
args = json.loads(function_call.arguments)
result = cactify_name(**args)
print(f"Result: {result}")

Result: Colinactus


Now that we have the output, we want to feed it back to the model to get a final response.

In [ ]:
# Add the model's call to our function to the input list
input_list += response_output
# Append the function call output to the input list
input_list.append(
    {"type": "function_call_output", "call_id": function_call.call_id, "output": result}
)
input_list

[{'role': 'user',
  'content': 'What would my name, Colin, be if it were cactus-ified?'},
 ResponseReasoningItem(id='rs_07760e000eb25cdc0068c9d79d790881a3a2054d2db49f34ae', summary=[], type='reasoning', content=None, encrypted_content=None, status=None),
 ResponseFunctionToolCall(arguments='{"name":"Colin"}', call_id='call_y8ouIrJZrcKo0BCIP8UCSuVK', name='cactify_name', type='function_call', id='fc_07760e000eb25cdc0068c9d79e50ac81a3b4273b683a1e7e1d', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_y8ouIrJZrcKo0BCIP8UCSuVK',
  'output': 'Colinactus'}]

In [16]:
response = client.responses.create(
    model="gpt-4o",
    instructions="Tell the user what their name would be if it were cactus-ified.",
    tools=tools,
    input=input_list,
)

In [17]:
rich_print(response.output[0].content[0])

ResponseOutputText(
    annotations=[],
    text='Your cactus-ified name would be "Colinactus"! 🌵',
    type='output_text',
    logprobs=[]
)

There we go! We've successfully used function calling with the Python SDK to cactify a name. It's a bit silly, but it shows how function calling can be used.

## Detecting function calls and saving history

We manually called the function and fed the output back to the model. In a real application, you'd want to automate this process. You could write a loop that checks if the model's response includes a function call, executes the function, and then sends the result back to the model until you get a final text response.

In [96]:
all_messages = []


def prompt(user_input: str) -> str:
    """Prompt the model with the user input."""

    # Add the user input to the conversation history
    all_messages.append({"role": "user", "content": user_input})
    # Prompt the model with the user input
    response = client.responses.create(
        model="gpt-5-nano",
        tools=tools,
        input=all_messages,
    )

    for event in response.output:
        all_messages.append(event)

        # There's a request from the model to use a tool
        if event.type == "function_call":
            function_name = event.name
            function_args = json.loads(event.arguments)

            # Execute the function based on its name
            if function_name == "cactify_name":
                result = cactify_name(function_args["name"])

            # Add the function call output to the all_messages list
            # Use the exact format expected by the API
            all_messages.append(
                {
                    "type": "function_call_output",
                    "call_id": event.call_id,
                    "output": json.dumps(result),
                }
            )

            # Now feed the function result back to the model
            final_response = client.responses.create(
                model="gpt-5-nano",
                instructions="Respond with the what the name would be if it were cactus-ified in a sentence.",
                tools=tools,
                input=all_messages,
            )

            for final_event in final_response.output:
                if final_event.type == "message":
                    text = final_event.content[0].text
                    all_messages.append({"role": "assistant", "content": text})
                    return text

        for event in response.output:
            if event.type == "message":
                text = event.content[0].text
                all_messages.append({"role": "assistant", "content": text})
                return text

In [97]:
print(prompt("What would my name, Colin, be if it were cactus-ified?"))

Colin would be Colinactus.


In [98]:
print(prompt("What about Simon?"))

Simon would be Simonactus.


In [100]:
print(prompt("What names did I ask about"))

You asked about Colin and Simon.
